In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import spacy
import torch

In [11]:
Rest_google = pd.read_csv('../Datasets/Restaurantes_Google.csv',delimiter=',')
Rest_google.head()

FileNotFoundError: [Errno 2] No such file or directory: '../Datasets/Restaurantes_Google.csv'

In [168]:
Rest_google = Rest_google[['name','address','latitude','longitude','category','description','MISC']]
Rest_google.head()

,name,address,latitude,longitude,category,description,MISC
0,Roux's Roadhouse 73,"Roux's Roadhouse 73, 784 WI-73, Nekoosa, WI 54457",44.306954,-89.845783,['Restaurant'],NaN,"{'Service options': ['Takeout', 'Dine-in', 'De..."
1,Crepes n' Tacos Mexican Grill,"Crepes n' Tacos Mexican Grill, suit #5, 5390, ...",36.164633,-115.060780,"['Mexican restaurant', 'Crêperie', 'Ice cream ...",NaN,"{'Service options': ['Delivery'], 'Amenities':..."
2,The Big Eazy,"The Big Eazy, 2053 N Doctor M.L.K. Jr Dr, Milw...",43.057658,-87.914643,"['Cajun restaurant', 'Creole restaurant']",NaN,"{'Service options': ['Delivery'], 'Amenities':..."
3,Subway,"Subway, 250 E Wolf Run, Mukwonago, WI 53149",42.854168,-88.314247,"['Sandwich shop', 'Caterer', 'Fast food restau...",Casual counter-serve chain for build-your-own ...,"{'Service options': ['Takeout', 'Dine-in'], 'D..."
4,Papa Kelsey's,"Papa Kelsey's, 165 E 1400 N, Logan, UT 84341",41.758094,-111.830290,['Pizza restaurant'],"Chain eatery serving baked subs, classic pizza...","{'Service options': ['Delivery'], 'Health & sa..."


In [169]:
Rest_google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3223 entries, 0 to 3222
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         3223 non-null   object 
 1   address      3219 non-null   object 
 2   latitude     3223 non-null   float64
 3   longitude    3223 non-null   float64
 4   category     3223 non-null   object 
 5   description  1061 non-null   object 
 6   MISC         3207 non-null   object 
dtypes: float64(2), object(5)
memory usage: 176.4+ KB


In [170]:
type(Rest_google['category'][0])

str

In [171]:
def eval_lower(lista):
    if lista:
        lista = eval(lista)
        for index,palabra in enumerate(lista):
            lista[index] = palabra.lower()
        return lista

In [172]:
Rest_google['category'] = Rest_google['category'].apply(eval_lower)

In [173]:
Rest_google['category']

0                                            [restaurant]
1       [mexican restaurant, crêperie, ice cream shop,...
2                   [cajun restaurant, creole restaurant]
3       [sandwich shop, caterer, fast food restaurant,...
4                                      [pizza restaurant]
                              ...                        
3218    [thai restaurant, takeout restaurant, vegetari...
3219               [chinese restaurant, asian restaurant]
3220                                 [mexican restaurant]
3221                                  [french restaurant]
3222    [restaurant, american restaurant, chicken rest...
Name: category, Length: 3223, dtype: object

In [174]:
type(Rest_google['category'][0])

list

In [175]:
Rest_google['MISC'][0]

'{\'Service options\': [\'Takeout\', \'Dine-in\', \'Delivery\'], \'Health & safety\': [\'Staff required to disinfect surfaces between visits\'], \'Highlights\': [\'Great beer selection\', \'Great cocktails\'], \'Popular for\': [\'Lunch\', \'Dinner\', \'Solo dining\'], \'Accessibility\': [\'Wheelchair accessible entrance\'], \'Offerings\': [\'Alcohol\', \'Beer\', \'Cocktails\', \'Coffee\', \'Comfort food\', \'Happy hour food\', \'Hard liquor\', "Kids\' menu", \'Quick bite\', \'Wine\'], \'Amenities\': [\'Bar onsite\', \'Good for kids\', \'High chairs\', \'Restroom\'], \'Atmosphere\': [\'Casual\', \'Cozy\'], \'Crowd\': [\'Groups\', \'Tourists\']}'

In [176]:
texto_corregido = Rest_google['MISC'][0].replace("\'","'")
texto_corregido

'{\'Service options\': [\'Takeout\', \'Dine-in\', \'Delivery\'], \'Health & safety\': [\'Staff required to disinfect surfaces between visits\'], \'Highlights\': [\'Great beer selection\', \'Great cocktails\'], \'Popular for\': [\'Lunch\', \'Dinner\', \'Solo dining\'], \'Accessibility\': [\'Wheelchair accessible entrance\'], \'Offerings\': [\'Alcohol\', \'Beer\', \'Cocktails\', \'Coffee\', \'Comfort food\', \'Happy hour food\', \'Hard liquor\', "Kids\' menu", \'Quick bite\', \'Wine\'], \'Amenities\': [\'Bar onsite\', \'Good for kids\', \'High chairs\', \'Restroom\'], \'Atmosphere\': [\'Casual\', \'Cozy\'], \'Crowd\': [\'Groups\', \'Tourists\']}'

In [177]:
eval(texto_corregido)

{'Service options': ['Takeout', 'Dine-in', 'Delivery'],
 'Health & safety': ['Staff required to disinfect surfaces between visits'],
 'Highlights': ['Great beer selection', 'Great cocktails'],
 'Popular for': ['Lunch', 'Dinner', 'Solo dining'],
 'Accessibility': ['Wheelchair accessible entrance'],
 'Offerings': ['Alcohol',
  'Beer',
  'Cocktails',
  'Coffee',
  'Comfort food',
  'Happy hour food',
  'Hard liquor',
  "Kids' menu",
  'Quick bite',
  'Wine'],
 'Amenities': ['Bar onsite', 'Good for kids', 'High chairs', 'Restroom'],
 'Atmosphere': ['Casual', 'Cozy'],
 'Crowd': ['Groups', 'Tourists']}

In [178]:
Rest_google['coord'] = Rest_google.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
Rest_google.drop(['latitude','longitude'],axis=1,inplace=True)

In [179]:
Rest_google.head()

,name,address,category,description,MISC,coord
0,Roux's Roadhouse 73,"Roux's Roadhouse 73, 784 WI-73, Nekoosa, WI 54457",[restaurant],NaN,"{'Service options': ['Takeout', 'Dine-in', 'De...","(44.3069541, -89.8457834)"
1,Crepes n' Tacos Mexican Grill,"Crepes n' Tacos Mexican Grill, suit #5, 5390, ...","[mexican restaurant, crêperie, ice cream shop,...",NaN,"{'Service options': ['Delivery'], 'Amenities':...","(36.1646332, -115.0607804)"
2,The Big Eazy,"The Big Eazy, 2053 N Doctor M.L.K. Jr Dr, Milw...","[cajun restaurant, creole restaurant]",NaN,"{'Service options': ['Delivery'], 'Amenities':...","(43.0576581, -87.914643)"
3,Subway,"Subway, 250 E Wolf Run, Mukwonago, WI 53149","[sandwich shop, caterer, fast food restaurant,...",Casual counter-serve chain for build-your-own ...,"{'Service options': ['Takeout', 'Dine-in'], 'D...","(42.8541679, -88.3142471)"
4,Papa Kelsey's,"Papa Kelsey's, 165 E 1400 N, Logan, UT 84341",[pizza restaurant],"Chain eatery serving baked subs, classic pizza...","{'Service options': ['Delivery'], 'Health & sa...","(41.7580941, -111.8302904)"


In [180]:
Rest_google['MISC'] = Rest_google['MISC'].replace('\'', "'")
Rest_google.head()

,name,address,category,description,MISC,coord
0,Roux's Roadhouse 73,"Roux's Roadhouse 73, 784 WI-73, Nekoosa, WI 54457",[restaurant],NaN,"{'Service options': ['Takeout', 'Dine-in', 'De...","(44.3069541, -89.8457834)"
1,Crepes n' Tacos Mexican Grill,"Crepes n' Tacos Mexican Grill, suit #5, 5390, ...","[mexican restaurant, crêperie, ice cream shop,...",NaN,"{'Service options': ['Delivery'], 'Amenities':...","(36.1646332, -115.0607804)"
2,The Big Eazy,"The Big Eazy, 2053 N Doctor M.L.K. Jr Dr, Milw...","[cajun restaurant, creole restaurant]",NaN,"{'Service options': ['Delivery'], 'Amenities':...","(43.0576581, -87.914643)"
3,Subway,"Subway, 250 E Wolf Run, Mukwonago, WI 53149","[sandwich shop, caterer, fast food restaurant,...",Casual counter-serve chain for build-your-own ...,"{'Service options': ['Takeout', 'Dine-in'], 'D...","(42.8541679, -88.3142471)"
4,Papa Kelsey's,"Papa Kelsey's, 165 E 1400 N, Logan, UT 84341",[pizza restaurant],"Chain eatery serving baked subs, classic pizza...","{'Service options': ['Delivery'], 'Health & sa...","(41.7580941, -111.8302904)"


In [181]:
for index,a in enumerate(Rest_google['MISC']):
    if isinstance(a,str):
        Rest_google['MISC'][index] = eval(Rest_google['MISC'][index])

In [182]:
Rest_google['MISC'][0]['Service options']

['Takeout', 'Dine-in', 'Delivery']

In [183]:
def misc(dicc):
    if isinstance(dicc,dict):
        lista = []
        for key in dicc.keys():
            for value in dicc[key]:
                lista.append(value.lower())
        return lista

Rest_google['MISC'] = Rest_google['MISC'].apply(misc)

In [184]:
Rest_google['MISC']

0       [takeout, dine-in, delivery, staff required to...
1                               [delivery, good for kids]
2                               [delivery, good for kids]
3       [takeout, dine-in, lunch, catering, debit card...
4       [delivery, reservations required, fireplace, l...
                              ...                        
3218    [outdoor seating, curbside pickup, no-contact ...
3219    [takeout, delivery, dine-in, mask required, lu...
3220    [delivery, takeout, mask required, lunch, dinn...
3221                                           [delivery]
3222    [takeout, dine-in, reservations required, fire...
Name: MISC, Length: 3223, dtype: object

In [185]:
Rest_google['keywords'] = Rest_google['category'] + Rest_google['MISC']

In [186]:
Rest_google.drop(['category','MISC','description'],axis=1,inplace=True)

In [187]:
Rest_google.head()

,name,address,coord,keywords
0,Roux's Roadhouse 73,"Roux's Roadhouse 73, 784 WI-73, Nekoosa, WI 54457","(44.3069541, -89.8457834)","[restaurant, takeout, dine-in, delivery, staff..."
1,Crepes n' Tacos Mexican Grill,"Crepes n' Tacos Mexican Grill, suit #5, 5390, ...","(36.1646332, -115.0607804)","[mexican restaurant, crêperie, ice cream shop,..."
2,The Big Eazy,"The Big Eazy, 2053 N Doctor M.L.K. Jr Dr, Milw...","(43.0576581, -87.914643)","[cajun restaurant, creole restaurant, delivery..."
3,Subway,"Subway, 250 E Wolf Run, Mukwonago, WI 53149","(42.8541679, -88.3142471)","[sandwich shop, caterer, fast food restaurant,..."
4,Papa Kelsey's,"Papa Kelsey's, 165 E 1400 N, Logan, UT 84341","(41.7580941, -111.8302904)","[pizza restaurant, delivery, reservations requ..."


In [188]:
Rest_google.to_parquet('../Datasets_ML/Rest_google.parquet')

***

In [189]:
Rest_yelp = pd.read_csv('../Datasets/Restaurantes_Yelp.csv',delimiter=',')
Rest_yelp.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
1,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,5505 S Virginia St,Reno,AB,89502,39.476117,-119.789339,2.5,339,1,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Restaurants, Italian","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
2,tMkwHmWFUEXrC9ZduonpTg,The Green Pheasant,215 1st Ave S,Nashville,TN,37201,36.159886,-86.773197,4.0,161,0,"{'RestaurantsGoodForGroups': 'True', 'HappyHou...","Restaurants, Japanese, Seafood","{'Wednesday': '16:0-22:0', 'Thursday': '16:0-2..."
3,2xVsWBNFwZOxIOdd9Mwnww,Cheeseburger In Paradise,116 N Pottstown Pike,Exton,PA,19341,40.029962,-75.630607,2.5,20,0,"{'NoiseLevel': ""u'average'"", 'HasTV': 'True', ...","Restaurants, Burgers",NaN
4,ljxNT9p0y7YMPx0fcNBGig,Tony's Restaurant & 3rd Street Cafe,312 Piasa St,Alton,IL,62002,38.896563,-90.186203,3.0,94,1,"{'RestaurantsReservations': 'True', 'Restauran...","Restaurants, Specialty Food, Steakhouses, Food...","{'Monday': '0:0-0:0', 'Tuesday': '16:0-21:30',..."


In [190]:
Rest_yelp = Rest_yelp[['name','address','latitude','longitude','categories','attributes']]
Rest_yelp.head()

,name,address,latitude,longitude,categories,attributes
0,St Honore Pastries,935 Race St,39.955505,-75.155564,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'RestaurantsDelivery': 'False', 'OutdoorSeati..."
1,Romano's Macaroni Grill,5505 S Virginia St,39.476117,-119.789339,"Restaurants, Italian","{'RestaurantsGoodForGroups': 'True', 'Restaura..."
2,The Green Pheasant,215 1st Ave S,36.159886,-86.773197,"Restaurants, Japanese, Seafood","{'RestaurantsGoodForGroups': 'True', 'HappyHou..."
3,Cheeseburger In Paradise,116 N Pottstown Pike,40.029962,-75.630607,"Restaurants, Burgers","{'NoiseLevel': ""u'average'"", 'HasTV': 'True', ..."
4,Tony's Restaurant & 3rd Street Cafe,312 Piasa St,38.896563,-90.186203,"Restaurants, Specialty Food, Steakhouses, Food...","{'RestaurantsReservations': 'True', 'Restauran..."


In [191]:
Rest_yelp['coord'] = Rest_yelp.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
Rest_yelp.drop(['latitude','longitude'],axis=1,inplace=True)

In [192]:
Rest_yelp.head()

,name,address,categories,attributes,coord
0,St Honore Pastries,935 Race St,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'RestaurantsDelivery': 'False', 'OutdoorSeati...","(39.9555052, -75.1555641)"
1,Romano's Macaroni Grill,5505 S Virginia St,"Restaurants, Italian","{'RestaurantsGoodForGroups': 'True', 'Restaura...","(39.4761165, -119.7893392)"
2,The Green Pheasant,215 1st Ave S,"Restaurants, Japanese, Seafood","{'RestaurantsGoodForGroups': 'True', 'HappyHou...","(36.1598858, -86.7731974)"
3,Cheeseburger In Paradise,116 N Pottstown Pike,"Restaurants, Burgers","{'NoiseLevel': ""u'average'"", 'HasTV': 'True', ...","(40.029962, -75.630607)"
4,Tony's Restaurant & 3rd Street Cafe,312 Piasa St,"Restaurants, Specialty Food, Steakhouses, Food...","{'RestaurantsReservations': 'True', 'Restauran...","(38.896563, -90.1862032987)"


In [193]:
Rest_yelp['categories'][0].replace(' ','')

'Restaurants,Food,BubbleTea,Coffee&Tea,Bakeries'

In [194]:
Rest_yelp['categories'][0].replace(' ','').lower().split(',')

['restaurants', 'food', 'bubbletea', 'coffee&tea', 'bakeries']

In [195]:
Rest_yelp['categories'] = Rest_yelp['categories'].apply(lambda text : text.replace(' ','').lower().split(','))

In [196]:
Rest_yelp.head()

,name,address,categories,attributes,coord
0,St Honore Pastries,935 Race St,"[restaurants, food, bubbletea, coffee&tea, bak...","{'RestaurantsDelivery': 'False', 'OutdoorSeati...","(39.9555052, -75.1555641)"
1,Romano's Macaroni Grill,5505 S Virginia St,"[restaurants, italian]","{'RestaurantsGoodForGroups': 'True', 'Restaura...","(39.4761165, -119.7893392)"
2,The Green Pheasant,215 1st Ave S,"[restaurants, japanese, seafood]","{'RestaurantsGoodForGroups': 'True', 'HappyHou...","(36.1598858, -86.7731974)"
3,Cheeseburger In Paradise,116 N Pottstown Pike,"[restaurants, burgers]","{'NoiseLevel': ""u'average'"", 'HasTV': 'True', ...","(40.029962, -75.630607)"
4,Tony's Restaurant & 3rd Street Cafe,312 Piasa St,"[restaurants, specialtyfood, steakhouses, food...","{'RestaurantsReservations': 'True', 'Restauran...","(38.896563, -90.1862032987)"


In [197]:
type(Rest_yelp['attributes'][0])

str

In [198]:
Rest_yelp['attributes'] = Rest_yelp['attributes'].replace('\'', "'")
Rest_yelp.head(1)

,name,address,categories,attributes,coord
0,St Honore Pastries,935 Race St,"[restaurants, food, bubbletea, coffee&tea, bak...","{'RestaurantsDelivery': 'False', 'OutdoorSeati...","(39.9555052, -75.1555641)"


In [199]:
for index,a in enumerate(Rest_yelp['attributes']):
    if isinstance(a,str):
        Rest_yelp['attributes'][index] = eval(Rest_yelp['attributes'][index])

In [200]:
Rest_yelp['attributes'][0].keys()

dict_keys(['RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BusinessParking', 'BikeParking', 'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'ByAppointmentOnly', 'WiFi', 'Alcohol', 'Caters'])

In [201]:
def keysflag(dicc):
    if isinstance(dicc,dict):
        lista = []
        for key in dicc.keys():
            if dicc[key] == 'True':
                lista.append(key.lower())
        return lista

Rest_yelp['attributes'] = Rest_yelp['attributes'].apply(keysflag)

In [202]:
Rest_yelp['attributes']

0                [bikeparking, restaurantstakeout, caters]
1        [restaurantsgoodforgroups, restaurantstakeout,...
2        [restaurantsgoodforgroups, happyhour, business...
3        [hastv, businessacceptscreditcards, outdoorsea...
4        [restaurantsreservations, outdoorseating, busi...
                               ...                        
15300    [restaurantstakeout, caters, hastv, goodforkid...
15301    [restaurantstakeout, restaurantsdelivery, busi...
15302    [bikeparking, businessacceptscreditcards, rest...
15303    [caters, businessacceptscreditcards, hastv, go...
15304    [wheelchairaccessible, caters, businessaccepts...
Name: attributes, Length: 15305, dtype: object

In [203]:
Rest_yelp['keywords'] = Rest_yelp['categories'] + Rest_yelp['attributes']

In [204]:
Rest_yelp.drop(['categories','attributes'],axis=1,inplace=True)

In [205]:
Rest_yelp.head()

,name,address,coord,keywords
0,St Honore Pastries,935 Race St,"(39.9555052, -75.1555641)","[restaurants, food, bubbletea, coffee&tea, bak..."
1,Romano's Macaroni Grill,5505 S Virginia St,"(39.4761165, -119.7893392)","[restaurants, italian, restaurantsgoodforgroup..."
2,The Green Pheasant,215 1st Ave S,"(36.1598858, -86.7731974)","[restaurants, japanese, seafood, restaurantsgo..."
3,Cheeseburger In Paradise,116 N Pottstown Pike,"(40.029962, -75.630607)","[restaurants, burgers, hastv, businessacceptsc..."
4,Tony's Restaurant & 3rd Street Cafe,312 Piasa St,"(38.896563, -90.1862032987)","[restaurants, specialtyfood, steakhouses, food..."


In [206]:
Rest_yelp.to_parquet('../Datasets_ML/Rest_yelp.parquet')

***

In [207]:
Rest_total = pd.concat([Rest_google, Rest_yelp], ignore_index=True)

In [208]:
Rest_total.head()

,name,address,coord,keywords
0,Roux's Roadhouse 73,"Roux's Roadhouse 73, 784 WI-73, Nekoosa, WI 54457","(44.3069541, -89.8457834)","[restaurant, takeout, dine-in, delivery, staff..."
1,Crepes n' Tacos Mexican Grill,"Crepes n' Tacos Mexican Grill, suit #5, 5390, ...","(36.1646332, -115.0607804)","[mexican restaurant, crêperie, ice cream shop,..."
2,The Big Eazy,"The Big Eazy, 2053 N Doctor M.L.K. Jr Dr, Milw...","(43.0576581, -87.914643)","[cajun restaurant, creole restaurant, delivery..."
3,Subway,"Subway, 250 E Wolf Run, Mukwonago, WI 53149","(42.8541679, -88.3142471)","[sandwich shop, caterer, fast food restaurant,..."
4,Papa Kelsey's,"Papa Kelsey's, 165 E 1400 N, Logan, UT 84341","(41.7580941, -111.8302904)","[pizza restaurant, delivery, reservations requ..."


In [209]:
type(Rest_total['keywords'][0])

list

In [210]:
Rest_total.to_parquet('../Datasets_ML/Rest_total.parquet')

***

In [2]:
df_total = pd.read_parquet('../Datasets_ML/Rest_total.parquet')
df_total.head()

,name,address,coord,keywords
0,Roux's Roadhouse 73,"Roux's Roadhouse 73, 784 WI-73, Nekoosa, WI 54457","[44.3069541, -89.8457834]","[restaurant, takeout, dine-in, delivery, staff..."
1,Crepes n' Tacos Mexican Grill,"Crepes n' Tacos Mexican Grill, suit #5, 5390, ...","[36.1646332, -115.0607804]","[mexican restaurant, crêperie, ice cream shop,..."
2,The Big Eazy,"The Big Eazy, 2053 N Doctor M.L.K. Jr Dr, Milw...","[43.0576581, -87.914643]","[cajun restaurant, creole restaurant, delivery..."
3,Subway,"Subway, 250 E Wolf Run, Mukwonago, WI 53149","[42.8541679, -88.3142471]","[sandwich shop, caterer, fast food restaurant,..."
4,Papa Kelsey's,"Papa Kelsey's, 165 E 1400 N, Logan, UT 84341","[41.7580941, -111.8302904]","[pizza restaurant, delivery, reservations requ..."


In [3]:
len(df_total)

18528

In [212]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18528 entries, 0 to 18527
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      18528 non-null  object
 1   address   18438 non-null  object
 2   coord     18528 non-null  object
 3   keywords  18335 non-null  object
dtypes: object(4)
memory usage: 579.1+ KB


In [213]:
model_name = "en_core_web_md"

def load_model(model_name): 
    try:
        nlp = spacy.load(model_name)
        print(f"Modelo {model_name} cargado exitosamente.")
        return nlp
    except OSError:
        print(f"El modelo {model_name} no está instalado. Descargando e instalando...")
        spacy.cli.download(model_name)
        nlp = spacy.load(model_name)
        return nlp
    
nlp_en = load_model(model_name)

Modelo en_core_web_md cargado exitosamente.


In [214]:
def df_embedding(lista):
    if isinstance(lista, np.ndarray):
        # Calcular embeddings solo si la lista es un numpy array
        embeddings_lista = np.array([nlp_en(palabra).vector for palabra in lista])

        # Si tienes GPU disponible y los embeddings de nlp_en están en GPU, mover los embeddings a GPU
        if torch.cuda.is_available() and isinstance(embeddings_lista, torch.Tensor):
            embeddings_lista = embeddings_lista.cuda()

        return embeddings_lista
    else:
        return None


In [215]:
df_total['embedding'] = df_total['keywords'].apply(df_embedding)

In [216]:
df_total.head()

,name,address,coord,keywords,embedding
0,Roux's Roadhouse 73,"Roux's Roadhouse 73, 784 WI-73, Nekoosa, WI 54457","[44.3069541, -89.8457834]","[restaurant, takeout, dine-in, delivery, staff...","[[0.84636, -1.1249, -2.3415, -0.12289, 3.0118,..."
1,Crepes n' Tacos Mexican Grill,"Crepes n' Tacos Mexican Grill, suit #5, 5390, ...","[36.1646332, -115.0607804]","[mexican restaurant, crêperie, ice cream shop,...","[[-1.15987, -0.23539498, -2.703, -0.27396, 4.7..."
2,The Big Eazy,"The Big Eazy, 2053 N Doctor M.L.K. Jr Dr, Milw...","[43.0576581, -87.914643]","[cajun restaurant, creole restaurant, delivery...","[[-1.72852, -3.57585, -0.14324999, -0.1046505,..."
3,Subway,"Subway, 250 E Wolf Run, Mukwonago, WI 53149","[42.8541679, -88.3142471]","[sandwich shop, caterer, fast food restaurant,...","[[-1.46857, -3.8650498, -1.97485, 2.0822, 1.41..."
4,Papa Kelsey's,"Papa Kelsey's, 165 E 1400 N, Logan, UT 84341","[41.7580941, -111.8302904]","[pizza restaurant, delivery, reservations requ...","[[0.32144, -1.7504499, -3.4772, 1.227705, 1.85..."


In [217]:
df_total.to_pickle('../Datasets_ML/Rest_final_ML.pickle')